In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
from torchvision import transforms
import torch.optim as optim 

transform_list = transforms.Compose([ transforms.ToTensor(), transforms.Normalize(mean=[0.0], std=[1.0]) ] )
train_set = datasets.MNIST(root='./data', train=True, download=True,  transform=transform_list)
test_set = datasets.MNIST(root='./data', train=False, download=True,  transform=transform_list)
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 14 * 14, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.pool(x)
        x = x.view(-1, 64 * 14 * 14)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

In [3]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=32, shuffle=True)
loss_criterion = nn.CrossEntropyLoss()
model = CNN().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
num_epochs = 30

for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
        outputs = model(inputs)
        optimizer.zero_grad()
        loss = loss_criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch + 1}. Loss: {loss.item()}")

Epoch: 1. Loss: 0.12528342008590698
Epoch: 2. Loss: 0.039659515023231506
Epoch: 3. Loss: 0.026629548519849777
Epoch: 4. Loss: 0.05391715466976166
Epoch: 5. Loss: 0.056548118591308594
Epoch: 6. Loss: 0.1684257686138153
Epoch: 7. Loss: 0.10250838100910187
Epoch: 8. Loss: 0.00401377584785223
Epoch: 9. Loss: 0.015404571779072285
Epoch: 10. Loss: 0.008531380444765091
Epoch: 11. Loss: 0.0016952321166172624
Epoch: 12. Loss: 0.006303559057414532
Epoch: 13. Loss: 0.014649441465735435
Epoch: 14. Loss: 0.018926307559013367
Epoch: 15. Loss: 0.06067979335784912
Epoch: 16. Loss: 0.021851198747754097
Epoch: 17. Loss: 0.09783076494932175
Epoch: 18. Loss: 0.0050385454669594765
Epoch: 19. Loss: 0.009141759015619755
Epoch: 20. Loss: 0.004731845110654831
Epoch: 21. Loss: 0.001507838605903089
Epoch: 22. Loss: 0.004757629707455635
Epoch: 23. Loss: 0.0012076770653948188
Epoch: 24. Loss: 0.0029919238295406103
Epoch: 25. Loss: 0.0007688119658268988
Epoch: 26. Loss: 0.0012865146854892373
Epoch: 27. Loss: 0.0043

In [4]:
model.eval()
total = 0
correct = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

acc =  correct/ total * 100
print(f"Test accuracy: {acc:.2f}%")

Test accuracy: 98.64%
